# Package imports

In [1]:
import pandas as pd
from utils.database_mysql import get_df_from_mysql, get_df_from_mongodb, redis_select
from utils.display_utils import colorprint

import warnings
warnings.filterwarnings('ignore')

# Queries Results

In [2]:
#QUERY 1
sql_query = "select * from Categories where Description not like '%sweet%'"
df = get_df_from_mysql(sql_query)
colorprint('\nInformations de toutes les catégories dont la description ne contient pas le mot "Sweet" :','green')
df

[20:57:16] Database connection to bigdata_mysql...
runtime -> 0.35 seconds

Informations de toutes les catégories dont la description ne contient pas le mot "Sweet" :


,CategoryID,CategoryName,Description,Picture
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales",b'\xff\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02...
1,4,Dairy Products,Cheeses,b'\xff\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02...
2,5,Grains/Cereals,"Breads, crackers, pasta, and cereal",b'\xff\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02...
3,6,Meat/Poultry,Prepared meats,b'\xff\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02...
4,7,Produce,Dried fruit and bean curd,b'\xff\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02...
5,8,Seafood,Seaweed and fish,b'\xff\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02...


In [3]:
#QUERY 2
#Get Customer
df = get_df_from_mongodb(collection_name='Orders',
                         mongo_condition= {'OrderID':10248},
                         normalized= True)
customerid = df['customer.CustomerID'].values[0]

#Get its information
df = get_df_from_mongodb(collection_name='Customers',
                         mongo_condition= {'ID':customerid},)

colorprint('\nCoordonées du client ayant passée la commande #10248 :','green')
df

[20:57:16] Database connection to bigdata_mongodb...
Nombre de documents collectés : 1
runtime -> 0.18 seconds
[20:57:16] Database connection to bigdata_mongodb...
Nombre de documents collectés : 1
runtime -> 0.16 seconds

Coordonées du client ayant passée la commande #10248 :


,_id,ID,Address,City,CompanyName,ContactName,ContactTitle,Country,Fax,Phone,PostalCode,Region
0,64391f7298a9f78edde8812a,VINET,59 rue de l'Abbaye,Reims,Vins et alcools Chevalier,Paul Henriot,Accounting Manager,France,26.47.15.11,26.47.15.10,51100,NULL


In [4]:
#QUERY 3
df = redis_select(redis_key_path = 'SHIPPERS:2:*', redis_key_map = 'Entity:ID:Info', redis_key_grouper = 'Entity:ID:Info',)
colorprint('\nInformations du transporteur #2 :','green')
df

[20:57:17] Database connection to bigdata_redis...
runtime -> 0.16 seconds

Informations du transporteur #2 :


Value
Entity   ID Info                       
SHIPPERS 2  COMPANYNAME  United Package
            PHONE        (503) 555-3199

In [5]:
#QUERY 4
#Get Employee
df = get_df_from_mongodb(collection_name='Employees',
                         mongo_condition= {'FirstName':'Margaret'},)
employeeid = int(df['EmployeeID'].values[0])

#Get its customers
df = get_df_from_mongodb(collection_name='Orders',
                         mongo_condition= {'EmployeeRef':employeeid},
                         normalized= True,)
customerids = df['customer.CustomerID'].unique().tolist()

#Get their information
df = get_df_from_mongodb(collection_name='Customers',
                         mongo_condition= {'ID':{'$in': customerids}},)
colorprint('\nInformations des clients ayant passée une commande encodée par Margaret :','green')
df

[20:57:17] Database connection to bigdata_mongodb...


Nombre de documents collectés : 1
runtime -> 0.16 seconds
[20:57:17] Database connection to bigdata_mongodb...
Nombre de documents collectés : 156
runtime -> 0.22 seconds
[20:57:17] Database connection to bigdata_mongodb...
Nombre de documents collectés : 75
runtime -> 0.2 seconds

Informations des clients ayant passée une commande encodée par Margaret :


,_id,ID,Address,City,CompanyName,ContactName,ContactTitle,Country,Fax,Phone,PostalCode,Region
0,64391f7298a9f78edde88082,ALFKI,Obere Str. 57,Berlin,Alfreds Futterkiste,Maria Anders,Sales Representative,Germany,030-0076545,030-0074321,12209,NULL
1,64391f7298a9f78edde88084,ANATR,Avda. de la Constitucin 2222,Mxico D.F.,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Mexico,(5) 555-3745,(5) 555-4729,05021,NULL
2,64391f7298a9f78edde88086,ANTON,Mataderos 2312,Mxico D.F.,Antonio Moreno Taquera,Antonio Moreno,Owner,Mexico,NULL,(5) 555-3932,05023,NULL
3,64391f7298a9f78edde88088,AROUT,120 Hanover Sq.,London,Around the Horn,Thomas Hardy,Sales Representative,UK,(171) 555-6750,(171) 555-7788,WA1 1DP,NULL
4,64391f7298a9f78edde8808a,BERGS,Berguvsvgen 8,Lule,Berglunds snabbkp,Christina Berglund,Order Administrator,Sweden,0921-12 34 67,0921-12 34 65,S-958 22,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...
70,64391f7298a9f78edde8812e,WARTH,Torikatu 38,Oulu,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Finland,981-443655,981-443655,90110,NULL
71,64391f7298a9f78edde88130,WELLI,"Rua do Mercado, 12",Resende,Wellington Importadora,Paula Parente,Sales Manager,Brazil,NULL,(14) 555-8122,08737-363,SP
72,64391f7298a9f78edde88132,WHITC,305 - 14th Ave. S. Suite 3B,Seattle,White Clover Markets,Karl Jablonski,Owner,USA,(206) 555-4115,(206) 555-4112,98128,WA
73,64391f7298a9f78edde88134,WILMK,Keskuskatu 45,Helsinki,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Finland,90-224 8858,90-224 8858,21240,NULL
